In [1]:
from dataclasses import dataclass
import numpy as np
from numba import njit, prange, get_thread_id
import numba
import os
import time
import math
from helper_functions import round
from levy_flights import estimate_mean_abs_levy, levy_step, levy_bit_flip,levy_jump_Q
from Swarm_IQ import decode_prod_plan, compute_objective, encode_prod_plan
from PSO import iterate, update_swarm

In [2]:

@dataclass(frozen=True)
class SwarmConfig:
    M: int = 8
    T: int = 8
    setup_costs: np.ndarray = np.array([112,184,144,187,127,147,100,188], dtype=np.float64)
    production_costs: np.ndarray = np.array([1,1,1,1,1,1,1,1], dtype=np.float64)
    inventory_costs: np.ndarray = np.array([5,3,4,6,5,3,6,2], dtype=np.float64)
    production_times: np.ndarray = np.array([1,1,1,1,1,1,1,1], dtype=np.float64)
    setup_times: np.ndarray = np.array([2,5,5,1,8,3,6,2], dtype=np.float64)
    capacities: np.ndarray = np.array([200,200,210,200,160,190,170,170], dtype=np.float64)
    # each row corresponds to one product
    demand: np.ndarray = np.array([[43.0, 29.0, 52.0, 0.0, 0.0, 0.0, 42.0, 0.0],
       [30.0, 0.0, 0.0, 40.0, 20.0, 0.0, 6.0, 27.0],
       [0, 20, 0, 50, 60, 11, 0, 30],
       [33.0, 43.0, 30.0, 0.0, 16.0, 48.0, 37.0, 33.0],
       [0.0, 0.0, 0.0, 41.0, 16.0, 0.0, 55.0, 0.0],
       [0, 0, 21, 13, 7, 0, 22, 0],
       [0.0, 25.0, 43.0, 25.0, 0.0, 52.0, 10.0, 42.0],
       [42.0, 18.0, 40.0, 2.0, 0.0, 71.0, 0.0, 41.0]], dtype=np.float64)


In [7]:
class Swarm:
    def __init__(self, n_particles, C1=1.5, C2=1.5, Vzmax=4.0, Vqmax=0.1, inertia = 1.0, stagn_thres = 49,
                 levy_alpha = 1.5, levy_c_frac = 0.1, levy_samples=100000, levy_q_max_step=1.0, target_avg_Q_step = 0.1):
        # keep the frozen config around for the other parameters
        self.cfg = SwarmConfig()
        N, M, T       = n_particles, self.cfg.M, self.cfg.T

        self.levy_samples = levy_samples
        self.levy_alpha   = levy_alpha
        self.levy_c_frac  = levy_c_frac
        self.levy_q_max_step = levy_q_max_step
        self.stagn_thres = stagn_thres

        # precompute mean |L| and scales
        mean_abs_L = estimate_mean_abs_levy(self.levy_alpha, self.levy_samples)
        self.levy_Sx = self.levy_c_frac  / mean_abs_L

        self.levy_Sq = target_avg_Q_step / mean_abs_L
        #self.levy_Sq = target_avg_Q_step

        self.inertia = inertia
        self.C1, self.C2, self.Vzmax, self.Vqmax = C1, C2, Vzmax, Vqmax

        # initialize all particles using 3D arrays
        self.Xs   = np.random.randint(0, 2, size=(N, T, M)).astype(np.float64)
        self.Qs = np.random.rand(N, T, M)
        self.VZs  = (np.random.rand(N, T, M)*2 - 1) * Vzmax
        self.VQs  = (np.random.rand(N, T, M)*2 - 1) * Vqmax

        self.pbest_X   = self.Xs.copy()
        self.pbest_Q   = self.Qs.copy()
        self.pbest_Val = np.full((N,  1, 2), np.inf)

        self.gbest_X   = np.zeros((T, M), dtype=np.float64)
        self.gbest_Q   = np.zeros((T, M), dtype=np.float64)
        self.gbest_Val = np.full((1,2), np.inf)
        self.stagn_count = np.zeros(N, dtype=np.int16)


        # one cold-start update to fill pbest and gbest
        self._reduce_global(update_swarm(
            self.Xs, self.Qs, self.VZs, self.VQs,
            self.pbest_X, self.pbest_Q, self.pbest_Val,
            self.gbest_X, self.gbest_Q,
            self.inertia, C1, C2, Vzmax, Vqmax,
            self.cfg.demand,
            self.cfg.production_times,
            self.cfg.setup_times,
            self.cfg.capacities,
            self.cfg.production_costs,
            self.cfg.setup_costs,
            self.cfg.inventory_costs,
            # no randomness needed for cold start: pass zeros
            np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N),
            np.zeros((N, T, M)), self.stagn_count, self.levy_Sx, self.levy_Sq, self.levy_alpha, 
            self.levy_q_max_step,  self.stagn_thres
        ))

    def _reduce_global(self, thread_results):
        # reduces all the thread-local minima into a single true global minimum
        values, indices = thread_results
        for t in range(values.shape[0]):
            i = indices[t]
            v0 = values[t, 0, 0]
            v1 = values[t, 0, 1]
            b0 = self.gbest_Val[0, 0]
            b1 = self.gbest_Val[0, 1]
            if (v0 < b0) or (v0 == b0 and v1 < b1):
                self.gbest_Val[0, 0] = v0
                self.gbest_Val[0, 1] = v1
                self.gbest_X[:] = self.pbest_X[i]
                self.gbest_Q[:] = self.pbest_Q[i]

    def optimize(self, n_iter):
        
        N, T, M = self.Xs.shape
    
        for t in range(n_iter):
           
 
            # pre-generate random numbers
            r1, r2, r3, r4 = (np.random.rand(N) for _ in range(4))
            rand_vals     = np.random.rand(N, T, M)

            # perform iteration of entire swarm
            thread_results = update_swarm(
                self.Xs, self.Qs, self.VZs, self.VQs,
                self.pbest_X, self.pbest_Q, self.pbest_Val,
                self.gbest_X, self.gbest_Q,
                self.inertia, self.C1, self.C2, self.Vzmax, self.Vqmax,
                self.cfg.demand, self.cfg.production_times,
                self.cfg.setup_times, self.cfg.capacities,
                self.cfg.production_costs, self.cfg.setup_costs, self.cfg.inventory_costs,
                r1, r2, r3, r4, rand_vals, self.stagn_count, self.levy_Sx, self.levy_Sq,
                self.levy_alpha, self.levy_q_max_step,  self.stagn_thres
            )
            # update global best solution
            self._reduce_global(thread_results)
    
        return decode_prod_plan(self.gbest_X.T, self.gbest_Q.T,self.cfg.demand) ,self.gbest_Val

        

In [8]:
#  pin a stable threading layer before Numba is loaded:
os.environ["NUMBA_THREADING_LAYER"] = "workqueue"
results = []
bad_results = []
for i in range(20):

    np.random.seed(i) 
    # instantiate swarm
    
    sw = Swarm(n_particles=100, C1=2.0, C2=2.0, Vzmax=4.0, Vqmax=0.05, inertia = 1.0, levy_c_frac = 0.1, levy_q_max_step=10.0, target_avg_Q_step = 0.1, stagn_thres = 23)
    # timing
    
    start = time.perf_counter()
    #plan, best_val = sw.optimize(n_iter=0)
    end = time.perf_counter()
    
    plan, best_val = sw.optimize(n_iter=10000)
    #X, Q = encode_prod_plan(plan.T,demand)
    #new_plan = decode_prod_plan(X.T,Q.T,demand)
    
    #if np.any(abs(plan - new_plan)>1):
        #print("true plan\n", plan)
        #print("encoded plan\n", new_plan)
        #print('-------------------------')
     
    #print(f"Elapsed time: {end - start:.4f} seconds")
    if best_val[0][0]==0.0:
        results.append(best_val[0][1])
   
    else:
        #print(i)
        bad_results.append((plan,best_val))

print(results)

[6548.0, 6548.0, 7092.0, 6686.0, 6534.0, 6913.0, 6992.0, 6724.0, 7036.0, 6673.0, 6548.0, 6603.0, 6963.0, 6788.0, 7014.0, 6686.0, 6608.0, 6734.0, 6779.0, 7026.0]


In [9]:
print(np.percentile(results,0))
print( np.percentile(results,5))
print( np.percentile(results,25))
print(np.percentile(results,50))
print( np.percentile(results,75))
print( np.percentile(results,90))
print(np.percentile(results,95))
print(np.percentile(results,100))

6534.0
6547.3
6606.75
6729.0
6970.25
7027.0
7038.8
7092.0


In [10]:
len(bad_results)

0